In [ ]:
#Produces some data that allows you to test the interactive chart without using data
from faker import Faker
fake = Faker()

In [ ]:
import requests
import json

def create_node():
    node = {}
    node["full_name"] = fake.name()
    node["email_address"] = fake.email()
    node["phone_numbers"] = fake.phone_number()
    node["id"] = fake.md5()
    node["summary"] = fake.text()
    node["job_title"] = fake.job()
    node["web_url"] = r"www.google.com"

    node["mugshot_url_template"] = "http://lorempixel.com/200/200"
    return node
root = create_node()


In [ ]:
root = create_node()
import numpy as np
import random

from scipy import stats

def create_random_children(node,depth):
    
    num_children = np.random.binomial(8-depth, 0.5, 1)[0]
    
    node["children"] = []
    
    for i in range(num_children):
        this_node = create_node()
        node["children"].append(this_node)
    
    for node in node["children"]:
        create_random_children(node, depth+1)
        
create_random_children(root,1)



In [ ]:

#ALso need data that for the select box
select_box = []


def add_to_select_box(node):

    new_item = {}
    new_item["id"] = node["id"]
    new_item["text"] =  node["full_name"] + " " +node["job_title"]
    select_box.append(new_item)
    
    if "children" in node:
        for c in node["children"]:
            add_to_select_box(c)


#Iterate through the nodes, adding to select box

add_to_select_box(root)    
select_box.sort(key=lambda x: x["text"])



In [ ]:


final_data = {"tree": root, "select_box":select_box}
    
import json
with open('../website/data/random_data.json', 'w') as outfile:
    json.dump(final_data, outfile)
